In [1]:
import spacy
from spacy import displacy
import glob
from tqdm import tqdm
import json

In [2]:
with open('train_data.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [3]:
with open('test_data.json', 'r', encoding='utf-8') as f:
    testing_data = json.load(f)

In [4]:
print(training_data[0])

{'text': "Nr. 7029 Konkurs-Ausschreibung. Im u . 20 der k. k. Forst⸗ und Domänen Direktion zu Wien ist eine Forstwartstelle mit dem Gehalte jährlicher 400 fl., der gesetzlichen Aktivitätszulage von 100 fl., dem systemmäßigen Deputatholze, eventuell auch mit dem Genusse eines Naturalquartiers zu besetzen. Der Bewerbungstermin für diesen Dienstposten wird auf fünf Wochen vom Tage der Verlautbarung dieser Konkurs⸗Ausschreibung im Verordnungsblatte des hohen k. k. Ackerbau⸗Ministeriums festgesetzt, und sind die eigen händig geschriebenen Bewerbungsgesuche von den im Sinne des Gesetzes vom 19. April 1872 (R. G. B. Nr. 60 ex 1872) anspruchsberechtigten Unteroffizieren mit den durch die Verordnung des Ministeriums für Landesvertheidigung vom 12. Juli 1872 Punkt 6 vorgeschriebenen Belegen und speziell mit dem Zeugnisse über die nach der Ministerial Verordnung vom 16. Jänner 1850 Nr. 63 R. G. B ab gelegte Prüfung für den Forstschutz⸗ und technischen Hilfs dienst und dem ärztlichen Zeugnisse übe

In [5]:
from spacy.tokens import DocBin
from spacy.util import filter_spans

nlp = spacy.blank('de')

In [6]:
train_doc_bin = DocBin()

for training_example in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    train_doc_bin.add(doc)

train_doc_bin.to_disk("train.spacy")

test_doc_bin = DocBin()

for training_example in tqdm(testing_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    test_doc_bin.add(doc)

test_doc_bin.to_disk("test.spacy")

 63%|████████████████████████████████████████████████▍                            | 934/1486 [00:00<00:00, 3183.81it/s]

Skipping entity
Skipping entity
Skipping entity


100%|██████████████████████████████████████████████████████████████████████████████| 637/637 [00:00<00:00, 4006.55it/s]

Skipping entity
Skipping entity
Skipping entity


In [7]:
## Generate a base_config file from spaCy's official documentation: https://spacy.io/usage/training

In [8]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python -m spacy train config.cfg --output ner --paths.train train.spacy --paths.dev test.spacy

[i] Saving to output directory: ner
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     36.83    6.24    3.33   49.65    0.06
  0     200         89.77   1261.51   74.42   79.68   69.81    0.74
  0     400        165.66    524.53   82.21   82.95   81.49    0.82
  1     600         67.56    378.08   83.62   84.48   82.78    0.84
  1     800        119.37    413.02   85.90   88.20   83.73    0.86
  2    1000        146.88    292.90   86.61   89.04   84.32    0.87
  3    1200        182.13    216.98   86.58   86.83   86.32    0.87
  4    1400        255.13    225.48   85.93   91.03   81.37    0.86
  6    1600        272.1

In [10]:
nlp_ner = spacy.load('ner/model-best')

In [11]:
total_annotations = 0
correct_matches = 0

for entry in testing_data:
    identified = []
    entities = nlp_ner(entry['text'])
    
    for entity in entities.ents:
        identified.append(str(entity.text))
        
    annotated_positions = [entry['text'][start:end] for start, end, _ in entry['entities']]
    total_annotations += len(annotated_positions)
    
    # Directly compare the annotated positions to identified positions
    identified_set = set(identified)
    annotated_set = set(annotated_positions)
    
    # Calculate the number of correct matches
    matches = len(identified_set & annotated_set)
    
    # Calculate the fraction of correct matches compared to total annotated entities
    match_fraction = matches / len(annotated_positions) if len(annotated_positions) > 0 else 0
    
    correct_matches += match_fraction
    
    # Print annotations and predictions for mismatches
    if match_fraction != 1:
        print(f"Annotations: {annotated_positions}")
        print(f"Predictions: {identified}")
        print()

Annotations: ['Krankenschwestern']
Predictions: []

Annotations: ['Cassier', 'Rechnungsführer', 'Hausadministrator', 'Staatsbeamter']
Predictions: ['Cassier', 'Rechnungsführer', 'Hausadministrator']

Annotations: ['Handelsangestellter']
Predictions: []

Annotations: ['Supplentenstelle']
Predictions: ['Suppleutenstelle']

Annotations: ['Kontrolorspostens', 'Amtsschreiberspostens']
Predictions: ['Verwalter']

Annotations: ['Maler', 'Anstreicher', 'Lackirer']
Predictions: ['Anstreicher', 'Lackirer']

Annotations: ['Kellnerin', 'Alleinkellnerin', 'Markörin']
Predictions: ['Kellnerin', 'Markörin']

Annotations: ['Magazineurs']
Predictions: ['Magazineurstelle']

Annotations: ['Geiger', 'Stehgeiger', 'Geigerin', 'Cellistin', 'Pianistin']
Predictions: ['Geigerin', 'Cellistin', 'Pianistin', 'Kapellmeister']

Annotations: ['Malergehilfen']
Predictions: []

Annotations: ['Schlosser', 'Kesselschmied', 'Vorarbeiter']
Predictions: ['Schlosser', 'Vorarbeiter']

Annotations: ['Turnlehrerstelle', 'Lehr

In [12]:
accuracy = correct_matches / len(testing_data) if testing_data else 0

print(f'Accuracy: {accuracy:.2%}')

Accuracy: 86.59%


In [13]:
# Test on some examples
doc = nlp_ner("Tüchtiger Gärtner wird aufgenommen.")
displacy.render(doc, style='ent', jupyter=True)

In [14]:
doc = nlp_ner("Fesche solide 12599 Kassierkellnerin und tüchtige Köchin finden sofort Stellung. Stadtparkrestaurant, Saaz.")
displacy.render(doc, style='ent', jupyter=True)

In [15]:
doc = nlp_ner("Avis f. deutsche stellensuchende. sofort werden plazirt nach Ungarn: 1 sekretär zu einem Grafen 1000 fl., 1 Güter⸗Inspektor 900 fl. und Tantiéme, 1 Oekonomiebeamter 800 fl. und Deputat, 2 Wirthschaftsadjunkten à 400 fl. pro anno und freie stazion, 1 Oberförster 1200 fl., 1 Magazineur 800 fl., 1 Buchhalter 1000 fl., 1 Brennereileiter 900 fl., 1 Braumeister 1000 fl., 1 Portier 700 fl., 1 ArbeitsAufseher 750 fl., 1 Fabriks⸗Aufseher in einer chemischen Fabrik 800 fl., 1 deutscher Erzieher 40 fl., 1 Reisebegleiterin 40 fl., 1 Hausrepräsentantin 35 fl. und 1 Gesellschafterin 30 fl. pro Monat und freie stazion, durch die Plazierungs⸗Agentur des B. Malík, Budapest, sommergasse 2. Anfragen werden nur gegen Einsendung von 3 stück Briefmarken beantwortet. 6540.")
displacy.render(doc, style='ent', jupyter=True)

In [16]:
doc = nlp_ner('Akkumulatorenfachmann,ſelbſtändiger Arbeiter, in Auto“, Radio. und Telephon⸗Arbeiten verſiert, ſucht ſeine Stelle zu verbeſſern. In⸗ oder Ausland. Sene und Tſchechiſch. 4 11. Angebote erbeten an Ernſt 114 Prag⸗Smichov, Nadraini 48²⁵')
displacy.render(doc, style='ent', jupyter=True)